# Applied Face Detection

## Copyright notice

This version (c) 2019 Fabian Offert, [MIT License](LICENSE).

## Imports

In [33]:
import sys
import os
sys.path.append('lib/')
from esu_util import show_img, np_PIL, PIL_np
import face_recognition
from tqdm import tqdm_notebook

Read image files in our dataset.

In [40]:
img_files = []
for root, dirs, files in os.walk('/home/fabian/datasets-live/moma/images'):
    for file in files:
        if file.endswith('.jpg'):
            img_files.append(f'{root}/{file}')
print(f'{len(img_files)} images found')
img_files = img_files[:100]

64780 images found


## Detect all faces and write to files

In [ ]:
savepoint = 0
border = 50 # Border around face in percent of face size
if not os.path.exists('faces_small'): os.makedirs('faces_small')

faces = 0
for n, path in enumerate(tqdm_notebook(img_files[savepoint:])):

    image = face_recognition.load_image_file(path)
    
    print(image.shape)
    
    face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="cnn")
    
    for face_nr, face_location in enumerate(face_locations):
        
        try:
                        
            top, right, bottom, left = face_location

            h = bottom-top
            w = right-left

            top -= int((h/100)*border)
            if top < 0: top = 0

            bottom += int((h/100)*border)
            if bottom > image.shape[0]: bottom = image.shape[0]

            left -= int((w/100)*border)
            if left < 0: left = 0

            right += int((w/100)*border)
            if right > image.shape[1]: bottom = image.shape[1]

            faces+=1
            file = path.split('/')[-1]
            face_file = f'faces_small/{file}_{face_nr+1}_{len(face_locations)}.jpg'
            pil_image = np_PIL(image[top:bottom, left:right])
            pil_image.save(face_file)
        
        except:
            continue
                
print(f'Found {faces} faces')